# Project 1: Assembling Genes

   <div class="alert alert-block alert-danger">
    <center>Due: <b>Monday, September 5, 8:59pm</b>.</center> 
   </div>
   
   <div class="alert alert-block alert-warning">
   <center>
       <b>Collaboration and Resource Policy</b>
    </center>
    
For this assignment, you are encouraged to work with one other person satisfying the constraints from Class 2. 
You are permitted (actually _encouraged_) to discuss these problems with anyone you want, including other students in the class. If you do discuss the specific questions in the assignment with anyone other than your assignment partner and the course staff, though, you should list them in the _External resources used_ section below.
    
You are welcome to use any resources you want for this assignment, other than ones that would defeat the purpose of the assignment. This means you should not look at answers or code from previous semesters of this course, or from any other students in the class (other than your collaboration with your partner), and if you find code that implements the problem you are being asked to do for the assignment, you should not use that code. You should document all external resource you use that are not part of the course materials in the _External resources used_ section below.
    </div>

**Team submitting this assignment:**  
<div class="alert alert-success">
    <b><em>list each member of your team here, including both your name and UVA computing id</em></b>

Team Members (Names): Emma Forrestal & Sarah Murphy

Team Member UVA Computing IDs: ecf5rcw & sem7zfj

</div>

**External resources used:** 
<div class="alert alert-success">
<em>It is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here.</em>
    
External Resources Used:

</div>

In this project, we will explore genome assembly—the process of determining the order of nucleotides in DNA from fragmented reads. As you might have studied in the reading assignments, genome assembly can get quite complicated, as problems such as full sequence coverage, finding a good length for reads (the $k$ in $k$-mer), and sequencing errors present challenges for sequencing analysis and accuracy. You can assume perfect coverage for all parts of the assignment and no read errors for the first two questions.


<b>Submission</b>: Please submit the code you wrote to generate your answers for all parts using this form: <a href="https://forms.gle/rNTXfYojTLEQ8idg6"><em>https://forms.gle/rNTXfYojTLEQ8idg6</em></a>. Your answers should be in the Jupyter Notebook, along with your code. Before submission, you should make a copy of your notebook file with the name <i>uvaid1\_uvaid2.ipynb</i> (where <i>uvaidn</i> is each teammates UVA id) so the submitted file identifies you. You and your partner should submit a single file once together. Submission is due 8:59 pm (EST) on Monday, September 5.

## Install basic required packages.

- Install basic required packages, should be run only once. You may need to restart the kernel after this stage.
- Make sure you have [graphviz](https://graphviz.org/download/) installed on your system.
- The second cell adds Graphviz to your path, you may have to change based on where the install folder is.

<b>NOTE: We provide utils.py, which may contain helpful functions for you to use, as well as gvmagic.py, which is a deprecated package to use graphviz within the notebook</b>

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
# os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
# os.environ["PATH"] += os.pathsep + '/Users/emmaforrestal/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/graphviz'
os.environ["PATH"] += os.pathsep + '/Users/emmaforrestal/opt/anaconda3/lib/python3.8/site-packages/graphviz'


## Genome Assembly

For this part, you're given reads generated while trying to sequence the DNA of a TeleTubby (some unknown organism) with a \textit{very} small genetic code. By answering the following questions, you will learn how to assemble the original genome sequence from sequence reads.

Sequencing data is often stored in FASTQ file format. In TeleTubby.fastq, you will find the data organized in a particular order that repeats every four lines. The first line contains the metadata that encodes the name of the read, the experiment type, the kind of sequencing machine used, etc. The second line is the sequence of bases. The third line functions as a placeholder line. The fourth line is a sequence of base qualities that encode the qualities for the corresponding bases in the sequence line. We will only work with the sequence and quality score lines in this question.

In [3]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import utils
from tqdm import tqdm

#### Question 1.1.1 GC-content

The GC-content (or the ratio of G and C nucleotides) is related to the melting temperature of the DNA double helix. Use the following equation to calculate the melting temperature of DNA for TeleTubby $t_m$ in Celsius:

\begin{equation*}
t_m = 64.9+0.41(\%GC)-\frac{500}{\text{length of sequence}}
\end{equation*}

As a reference, the human genome is known to have between 35%-60% GC-content. 

In [4]:
# Read sequence reads (error-free) from file
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
# print(sequence_reads)
# print(qualities)
# Calculate %GC content
gc_count = 0
total_count = 0
for seq in sequence_reads:
    n = len(seq)
    total_count = total_count + n
    for let in seq:
        if let == "C" or let == "G":
            gc_count = gc_count + 1
gc_perc = (gc_count * 100) / total_count
# Print out temperature in Celsius
temp = 64.9 + (0.41 * gc_perc)
temp = temp - (500 / total_count)
print(round(temp,2), "degrees Celsius")

84.35 degrees Celsius


#### Question 1.1.2 Interpreting quality scores

Phred33 quality scores are represented as the character with an ASCII code equal to its value + 33 (to make them easy to print alongside genome sequences). List the top 5 most frequent scores in ASCII symbol as well as their Phredd33 scores in TeleTubby.fastq. You can refer to the [official Illumina website](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) to reference the scoring encoding.

What is the average Phred33 score in TeleTubby.fastq?

In [5]:
# Calculate and print average Phred33 score
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
# print(qualities)
total_phreds = 0
counts = dict()
num = len(qualities)

for score in qualities:
    for char in score:
        phred33 = ord(char) - 33
        total_phreds += phred33
        if char in counts.keys():
            counts[char] = counts.get(char) + 1
        else:
            counts[char] = 1;

avg_phred = total_phreds / num
print ("The average Phredd33 score is", round(avg_phred,2))

# print(counts)
sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse = True))
# print(sorted_counts)

keys = list(sorted_counts.keys())
# print(keys)

print("The most frequent score is ", ord(keys[0]) - 33, "and this character is", keys[0])
print("The 2nd most frequent score is ", ord(keys[1]) - 33, "and this character is", keys[1])
print("The 3rd most frequent score is ", ord(keys[2]) - 33, "and this character is", keys[2])
print("The 4th most frequent score is ", ord(keys[3]) - 33, "and this character is", keys[3])
print("The 5th most frequent score is ", ord(keys[4]) - 33, "and this character is", keys[4])

The average Phredd33 score is 275.81
The most frequent score is  20 and this character is 5
The 2nd most frequent score is  30 and this character is ?
The 3rd most frequent score is  35 and this character is D
The 4th most frequent score is  42 and this character is K
The 5th most frequent score is  37 and this character is F


#### Question 1.1.3 Frequency analysis

Looking at repetitions in the sequence can be helpful in estimating the "redudancy" in the organisms. Humand and other evolved animals have a lot of redundancy, while smaller organisms like bacteria have highly packed genomes. One heuristic to estimate this before actually performing the assembly could be looking at how often certain $k$-mers are repeated.

<b>Print out the 3 most frequent k-mers with their frequencies</b>

In [6]:
# Find and print out the three most repeated k-mers and their frequencies
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')

# print(sequence_reads)
counts = dict()
    
for seq in sequence_reads:
    num = len(sequence_reads)
    if seq in counts.keys():
        counts[seq] = counts.get(seq) + 1
    else:
        counts[seq] = 1;
            
# print(counts)

sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse = True))
# print(sorted_counts)

keys = list(sorted_counts.keys())

first = keys[0]
print("The most repeated k-mer is", first, "and the frequency is", sorted_counts[first])
second1 = keys[1]
second2 = keys[2]
second3 = keys[3]
print("The second most repeated k-mers is a three way tie between",second1,",",second2,",","and",second3, "and the frequency for these is", sorted_counts[second1])

print("The third most repeated k-mer is a tie between all the remaining k-mers which all occur with a frequency of 1")

The most repeated k-mer is GCTATCGC and the frequency is 3
The second most repeated k-mers is a three way tie between CGCTATCG , TATCGCAA , and CTATCGCA and the frequency for these is 2
The third most repeated k-mer is a tie between all the remaining k-mers which all occur with a frequency of 1


### Question 1.2. Greedy approach

One of the approaches to assemble the genome from the given reads is a greedy algorithm. Have a look at the greedy algorithm described on [Wikipedia](https://en.wikipedia.org/wiki/Sequence_assembly#Greedy_algorithm) and answer the following.

#### Question 1.2.1 What would the runtime be of this algorithm, given $n$ $k$-mer reads?

<i>Answer</i>: The greedy algorithm contains an external while loop that runs through all the fragments, which takes n time. There is also a pairwise alignment calculation which has a run time of O(n^2) because each fragment being paired has to be compared with every other fragment. This goes through the while loop n times. Therefore the runtime with this algorithm would be O(n^3).

#### Question 1.2.2 Would this algorithm always yield a unique solution?

<i>Answer</i>: This algorithm is not going to always find a unique solution because there can be repetitions in the k-mers within the genome, as the genome is known to contain lengthy repeat sequences. This would ultimately lead to multiple solutions to the problem that are not always unique.

#### Question 1.2.3 Would this algorithm always yield the <i>right</i> solution?

<i>Answer</i>: This algorithm is not going to always find the right solution for the genome sequence. It cannot guarantee that it will find the shortest sequence of all the fragments, or that the shortest sequence is the correct sequence. Additionally since it is possible for there not to be an unique solution with every iteration of the algorithm, you cannot know which solution is the right solution. Perfect alignment and overlapping of fragments is not guaranteed with this algorithm, so you cannot guarantee the right solution.

### Question 1.3 Graph-based approaches

Graphs for genome assembly can be constructed in two ways:

- de Bruijn graph: Processing $k-$mers as nodes, with $(k-1)-$mers as edges, and
- Overlap graph: Processing $k-$mers as edges, with $(k-1)-$mers as nodes.

de Bruijn graphs can be processed to find Euler paths, while Overlap graphs can be processed to find Hamiltonian paths. Both of these are valid ways to reconstruct the original genome.

<b>Use one of these two techniques to reconstruct the sequence, and print out your reconstructed sequence. Which method did you pick out of the two, and why? (hint: imagine what would happen when we have millions of reads). Use the k-mers provided in TeleTubby.fastq</b>.

We provide some skeleton code that you may use, but you may also come up with your own solution.

In [4]:
# Read reads into graph

def build_graph(k_mers):
    edges = []
    nodes = set()
    # Your code here
    for k_mer in k_mers:
        size = len(k_mer)
        for n in range(0, size - (size - 1)):
            node = k_mer[n:n+(size - 1)]
            nodes.add(node)
        edge = k_mer[0:size]
        edge_add = [edge[0:(size-1)], edge[1:size]]
        edges.append(edge_add)
    return nodes, edges


sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
nodes, edges = build_graph(sequence_reads)
print(len(nodes))
print(len(edges))

285
293


In [ ]:
# Implement assembly algorithm

In [ ]:
# Print assembled sequence

In [5]:
# Needed to properly visualize graphs
%load_ext gvmagic

In [12]:
# Helper function to visualize de Bruijn graphs
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
nodes_bruijn, edges_bruijn = build_graph(sequence_reads)
%dotstr utils.viz_debruijn(nodes_bruijn, edges_bruijn)

FileNotFoundError: [Errno 2] No such file or directory: 'dot'

In [139]:
# Main assembly algorithm

def assemble_sequence(nodes, edges):
    assembled_sequence = ""
    # identifying starting node - each k mer without last letter was in nodes - never the 2nd part of edge tuple
    starting_node = ""
    second_half_node = []
    for edge in edges:
        second_half_node.append(edge[1])
    for node in nodes:    
        if node not in second_half_node:
            starting_node = node
            
    # use a stack, add to sequence in reverse order
    # push onto stack - once find dead end starting popping off stack and writing sequence
        #build stack
    stack = []
    
    #add start node to stack
    stack.append(starting_node)
    
    while len(stack) != 0:
        dead_end = True
        for edge in edges:
            if stack[len(stack)-1] == edge[0]:
                edges.remove(edge)
                stack.append(edge[1])
                dead_end = False
        if dead_end:
            found = stack.pop(len(stack)-1)
            assembled_sequence += found[len(found)-1]
                
    #reversing the sequence
    assembled_sequence = assembled_sequence[::-1]
    
    #adding in letter from starting node
    assembled_sequence = starting_node[:len(starting_node)-1] + assembled_sequence
        
    return assembled_sequence

# trying dictionary implementation
    
#     # dictionary - node --> second half of tuple
#     map = dict()
#     for edge in edges:
#         if edge[0] not in map:
#             new = []
#             new.append(edge[1])
#             map[edge[0]] = new
#         else:
#             options = map.get(edge[0])
#             options.append(edge[1])
#             map[edge[0]] = options
# #     print(map)
    
#     # identifying starting node - each k mer without last letter was in nodes - never the 2nd part of edge tuple
#     starting_node = ""
#     second_half_node = []
#     for edge in edges:
#         second_half_node.append(edge[1])
#     for node in nodes:    
#         if node not in second_half_node:
#             starting_node = node 
    
#     assembled_sequence += starting_node
#     # follow edge, add last letter to sequence
#     continue_walk = True
#     current_node = starting_node
#     next_sequences = map[current_node]
# #     print(next_sequences)
# #     print(next_sequences[0])

#     while continue_walk:
#         if len(next_sequences) != 1:
#             next_seq = next_sequences[len(next_sequences) - 1]
#         else:
#             next_seq = next_sequences[0]
#         assembled_sequence = assembled_sequence + next_seq[len(next_seq)-1:len(next_seq)]
# #         print(assembled_sequence)
#         # take out of dictionary (override) , if empty then done
#         next_sequences.remove(next_seq)
#         map[current_node] = next_sequences
#         # make the current node the next sequence
#         current_node = next_seq
#         #new next sequences
#         if current_node in map.keys():
#             next_sequences = map[current_node]
#         else:
#             continue_walk = False
# #             print(map)

In [140]:
# Output assembled sequence
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
nodes, edges = build_graph(sequence_reads)

assembled_seq = assemble_sequence(nodes, edges)
print(assembled_seq)
print("length: ",len(assembled_seq))

TACGCCAAATAGCAATGCGCAGGATAACAACTTATGTACTACATGTTGTTTCTCGTGCCCGCCAATGTCGAGAGATTTGTGCTATCGCTGCCCGGAGGCGCTATCGCAAACCGACTGTCGGACTCTTTCATGAGCAAAAAAAGTGGGAGTATGGTGCACATCCGCTATCGCAACCTAAGAGAGAAGGGGTTTTGTGTTAGCAGTTTCTTCATGCATCTCTTTACAAGAATTACAGGAGCCAAACACTCGCTGTCATGGTATCGACATATCGCTACTGGTGCCGCCCTTCGATGCAATGTT
length:  300


## Question 2 - Sequencing SARS-CoV-2 virus

Let's move on from TeleTubbies to real-world organisms. Let's start small- with a variant of the SARS-CoV-2 virus. You're given reads from <i>actual</i> genome sequencing runs in the SARS-CoV2.fastq file provided.

Repeat Question 1.3 on this data. You can re-use your implementation and simply run it on the new data. Print out your reconstructed sequence to a file "output.txt". For this part, we will still assume that all the reads are error-free. Set $k=25$.

In [141]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')

In [142]:
# Read reads into graph
nodes_covid, edges_covid = build_graph(sequence_reads_covid)

In [144]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')
# Read reads into graph
nodes_covid, edges_covid = build_graph(sequence_reads_covid)
# Call main assembly algorithm
assembled_covid_seq = assemble_sequence(nodes_covid, edges_covid)
print(assembled_covid_seq)
print(len(assembled_covid_seq))

...


In [146]:
# Write assembled sequence to file

assembled_seq = assembled_covid_seq # Use your assembled genome
with open("covid_overlap.txt", "w") as f:
    f.write(assembled_seq)

# Question 3- Error-Aware Assembly (Extra Credit)

In the parts above, we assumed error-free reads while assembling $k$-mers. As much as we'd like that, actual reads can (and do) have errors, captured by their Phred scores. For this question, you're given raw, actual reads from sequencing runs (download reads here: https://sra-pub-sars-cov2.s3.amazonaws.com/sra-src/SRR11528307/ABS2-LN-R1_cleaned_paired.fastq.gz).  Given these reads and their Phred33 scores, can you assemble the genome?

<b>Print out your assembled sequence, along with a brief explanation of how your algorithm works</b>

This is an open-ended question. You are free to use any approach to deal with the issue. Make sure you provide your code, along with any assumptions you may have, in the cells below.